In [2]:
import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv("vectorbt_reports/growing_master_backtest_stats.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=['Sharpe Ratio'])
df = df.drop(columns=['Start Value', 'End Value'])
#df = df[df["asset"]=="NVDA"]
#display(df[df["asset"]=="NVDA"])

In [44]:
returns = df['Total Return [%]'] / 100
cumulative_return = (1 + returns).prod() - 1
N = len(returns)  # number of periods
annualized_return = (1 + cumulative_return)**(12 / N) - 1
annualized_volatility = returns.std() * np.sqrt(12)
sharpe_annual = annualized_return / annualized_volatility
print(sharpe_annual)

-0.3486115960689809


In [45]:
def compute_annualized_sharpe(df):
    results = []

    for asset, group in df.groupby("asset"):
        # Convert returns from % to decimals
        returns = group['Total Return [%]'] / 100
        benchmark_returns = group['Benchmark Return [%]'] / 100
        N = len(returns)

        if N < 2:
            continue  # Skip assets with insufficient data

        # Strategy Sharpe
        cumulative_return = (1 + returns).prod() - 1
        annualized_return = (1 + cumulative_return)**(12 / N) - 1
        annualized_volatility = returns.std() * np.sqrt(12)
        sharpe_strategy = annualized_return / annualized_volatility if annualized_volatility > 0 else np.nan

        # Benchmark Sharpe
        cumulative_bench = (1 + benchmark_returns).prod() - 1
        annualized_bench_return = (1 + cumulative_bench)**(12 / N) - 1
        annualized_bench_vol = benchmark_returns.std() * np.sqrt(12)
        sharpe_benchmark = annualized_bench_return / annualized_bench_vol if annualized_bench_vol > 0 else np.nan

        results.append({
            'asset': asset,
            'strategy_sharpe': sharpe_strategy,
            'benchmark_sharpe': sharpe_benchmark
        })

    return pd.DataFrame(results)

In [46]:
sharpe_df = compute_annualized_sharpe(df)
display(sharpe_df.sort_values(by='strategy_sharpe', ascending=False))

,asset,strategy_sharpe,benchmark_sharpe
14,TSLA,1.188028,0.965867
13,NVDA,0.245811,-0.231651
3,BTC=F,-0.161315,0.580419
9,GC=F,-0.169814,2.864663
10,GOOGL,-0.348080,-0.506734
1,AMZN,-0.459030,0.382994
2,BTC-USD,-0.648146,0.654541
0,AAPL,-1.036865,0.652176
12,MSFT,-1.165879,-0.472800
4,CHFUSD=X,-1.280169,0.433831


In [3]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats.csv")
display(df)


,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-04-01,2025-04-01,252 days,10000.0,10343.869639,3.438696,31.265073,100.0,562.549216,11.999025,...,-2.127338,8 days 07:23:04.615384615,2 days,1.109735,12.735913,0.336712,0.418268,1.066014,0.473281,AAPL
1,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,8105.127205,-18.948728,6.188869,100.0,390.451653,23.341023,...,-2.118499,10 days 04:48:00,5 days 09:52:56.470588235,0.416157,-86.130582,-1.162486,-1.124014,0.812444,-1.517935,AMZN
2,2024-04-01 00:00:00,2025-04-01 00:00:00,366 days 00:00:00,10000.0,10492.785729,4.927857,22.190167,100.0,654.087593,24.852932,...,-3.387461,8 days 18:27:41.538461538,3 days 14:40:00,1.075332,15.896314,0.310086,0.197726,1.066259,0.492627,BTC-USD
3,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9895.775148,-1.042249,21.270247,100.0,510.248669,26.486976,...,-4.041845,7 days 21:36:00,2 days 04:48:00,0.983617,-4.168994,0.177746,-0.056638,1.036520,0.289160,BTC=F
4,2024-04-01 00:00:00,2025-03-31 00:00:00,260 days 00:00:00,10000.0,9222.932275,-7.770677,2.404336,100.0,652.157343,9.923887,...,-0.591627,11 days 12:00:00,2 days 01:50:46.153846153,0.476133,-22.854933,-1.492585,-1.081727,0.771635,-2.052103,CHFUSD=X
5,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,8065.237992,-19.347620,-14.944454,100.0,443.526268,26.578360,...,-2.298782,8 days 03:25:42.857142857,3 days 16:00:00,0.476296,-77.390480,-1.113229,-1.003473,0.810308,-1.493862,CL=F
6,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9010.408120,-9.895919,-0.723666,100.0,491.868005,10.151447,...,-0.795697,13 days 09:36:00,2 days 13:42:51.428571428,0.376154,-38.061226,-2.564931,-1.374978,0.625105,-3.069176,DX-Y_NYB
7,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9083.836118,-9.161639,-0.777941,100.0,455.175411,9.437452,...,-0.918567,11 days 20:00:00,2 days 22:40:00,0.419041,-38.173495,-2.318984,-1.371622,0.653372,-2.800513,DX=F
8,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9483.797804,-5.162022,7.162079,100.0,497.376748,8.611654,...,-1.101312,10 days 16:00:00,3 days 07:30:00,0.706075,-20.648088,-0.674109,-0.854802,0.889462,-0.867778,ES=F
9,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,10624.133921,6.241339,39.454500,100.0,494.708250,13.037942,...,-1.439631,9 days 03:41:32.307692307,2 days 00:00:00,1.361461,24.965357,0.659602,0.700056,1.123194,0.877670,GC=F
